In [5]:
# !pip install neuralprophet[live] # jupyter notebook 용

In [1]:
import os
import numpy as np
import pandas as pd

from neuralprophet import NeuralProphet, set_random_seed 
import neuralprophet

In [2]:
# 난수 생성기가 항상 일정한 값을 출력하게 하기 위해 seed 고정
random_seed = 2022
np.random.seed(random_seed)
set_random_seed(random_seed)

In [3]:
PATH = '/content/drive/MyDrive/Colab Notebooks/traffic forecasting/Project1/traffic forecasting'

In [4]:
train = pd.read_csv(f'{PATH}/data/train.csv')
val = pd.read_csv(f'{PATH}/data/validate.csv')
test = pd.read_csv(f'{PATH}/data/test.csv')
submission = pd.read_csv(f'{PATH}/data/sample_submission.csv')

In [5]:
train = pd.concat([train, val.loc[168:]], ignore_index=True)

In [6]:
336/2

168.0

In [7]:
train

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200101,0,83247,19128,2611,5161,1588,892,32263,1636,7938,1894,14957,3527,11174,2930,8640,14175,2035,5231,18492,48520,15393,16628,8804,2790,38657,1311,3482,11299,7072,1176,3810,748,3920,2133,3799
1,20200101,1,89309,19027,3337,5502,1650,1043,35609,1644,9896,1967,14682,3420,13884,3062,8349,15654,2417,5483,15283,57904,15834,19785,8483,3381,45759,1162,3849,13180,8771,1283,3763,782,3483,2057,4010
2,20200101,2,66611,14710,2970,4631,1044,921,26821,1104,9343,1838,11120,3587,17141,3255,7025,12520,1859,5058,14327,53876,13866,24501,8167,2670,27425,768,2299,7986,5426,1536,3229,491,2634,1526,3388
3,20200101,3,53290,13753,2270,4242,1021,790,21322,909,10009,1630,9836,3842,20715,2661,7226,9821,1724,4863,12787,51615,13528,28695,9459,2325,18893,632,1716,5703,3156,1104,2882,431,2488,1268,3686
4,20200101,4,52095,17615,2406,3689,1840,922,22711,1354,15497,1264,10668,4559,28308,2894,8201,9038,960,5166,11479,47424,18156,26977,10944,3583,18135,875,2421,5816,2933,1206,2433,499,2952,1927,5608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3442,20200524,19,314226,98345,10625,28618,8316,6684,141675,6619,23535,8572,67748,16791,33325,19124,42183,63229,13163,35383,73188,100849,84286,26458,15572,10341,139827,8254,16118,23304,14082,8447,21694,2180,15746,10903,21014
3443,20200524,20,300001,87871,8226,22706,6981,5743,142933,6295,20953,7851,61609,14119,30216,18610,35556,59070,12788,30675,68255,95107,76441,24140,13340,8872,142469,5225,15297,21919,14526,7332,19732,1990,14096,10028,17787
3444,20200524,21,304150,71126,6002,18317,4939,3779,133110,4781,15242,6101,48336,11534,25710,15651,27467,53787,12019,26317,61169,86434,61136,21076,9317,6392,138816,4072,12685,21135,14403,5443,16967,1359,11670,7963,14041
3445,20200524,22,236751,44947,3575,11455,3135,2536,98582,3267,8849,4553,33607,8069,18293,9723,16602,38615,8703,17694,47609,58492,37501,12090,5901,3938,100767,2489,8093,14427,10914,3861,11397,859,7270,5194,8230


In [8]:
data = pd.DataFrame()
data['ds'] = pd.to_datetime(train['날짜'].astype('str') +'-' + train['시간'].astype('str'),format="%Y%m%d-%H")

In [9]:
train.columns[2:]

Index(['10', '100', '101', '120', '121', '140', '150', '160', '200', '201',
       '251', '270', '300', '301', '351', '352', '370', '400', '450', '500',
       '550', '600', '650', '652', '1000', '1020', '1040', '1100', '1200',
       '1510', '2510', '3000', '4510', '5510', '6000'],
      dtype='object')

In [13]:
pred = pd.DataFrame(columns=['ds', 'y'])
pred['ds'] = pd.to_datetime(submission['timestamp'],format="%Y%m%d_%H")

In [14]:
pred

,ds,y
0,2020-05-25 00:00:00,NaN
1,2020-05-25 01:00:00,NaN
2,2020-05-25 02:00:00,NaN
3,2020-05-25 03:00:00,NaN
4,2020-05-25 04:00:00,NaN
...,...,...
163,2020-05-31 19:00:00,NaN
164,2020-05-31 20:00:00,NaN
165,2020-05-31 21:00:00,NaN
166,2020-05-31 22:00:00,NaN


In [15]:
for col in train.columns[2:]:
    data['y'] = train[col]
    m = NeuralProphet()
    metrics = m.fit(data, freq="H")
    forecast = m.predict(pred)
    submission[col] = forecast['yhat1']

INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 133


  0%|          | 0/276 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 5.75E-02, min: 9.35E-01


  0%|          | 0/276 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 4.95E-02, min: 1.17E+00


...

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 3.95E-02, min: 1.09E+00


  0%|          | 0/276 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 4.59E-02, min: 9.35E-01


  0%|          | 0/276 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 4.26E-02, min: 1.36E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 4.26E-02
Epoch[133/133]: 100%|██████████| 133/133 [00:31<00:00,  4.18it/s, SmoothL1Loss=0.00636, MAE=2.23e+3, RMSE=3.05e+3, RegLoss=0]


In [16]:
submission

,timestamp,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200525_0,81443.112199,17526.047997,1042.655151,6173.898438,905.330017,1060.516968,32756.785734,1216.442749,4645.801758,1567.247681,12747.650099,1911.697266,8456.649414,3419.688965,7181.523926,10372.364302,3759.978516,7587.517090,21993.385574,14853.798258,12615.214413,4158.855469,1606.263794,1400.291382,28306.506344,603.993103,3369.747070,-1271.770386,3936.201172,1771.755981,4793.286133,-75.687431,2827.297363,1918.079102,2886.219727
1,20200525_1,64108.751753,14302.269876,852.457764,5321.954590,702.575806,870.805969,25670.964563,978.103577,4107.761230,1320.204224,9838.266021,1309.026733,7012.690430,2918.634277,5627.392578,8297.636310,3421.172363,7118.052246,17497.265279,10858.521763,10317.284386,3032.906006,959.219360,1284.408325,21702.049131,454.373505,2960.471436,-2253.548340,2862.092041,1476.452637,4010.416748,-199.609650,2574.620850,1748.759644,2579.624268
2,20200525_2,58356.641042,15107.422754,976.925415,5250.567383,893.615601,968.724548,21339.304332,956.742432,4040.791748,1546.741333,9796.565127,1391.617676,6656.579102,3001.183105,5377.792480,7755.418716,3153.663574,7558.243164,15205.936684,9123.616254,10597.348332,2416.474365,850.383667,1463.212158,17250.782696,798.924377,2938.676025,-2986.842529,2195.347412,1327.977539,4261.008789,-53.569256,3324.927734,2052.674072,3110.754395
3,20200525_3,51666.604223,14704.705998,728.147278,5071.707031,556.096008,844.169006,18357.745440,852.377808,3611.309570,1682.067993,8527.878198,1335.270508,6632.655273,2700.916748,4868.639648,6704.273701,2917.107422,7621.581543,14779.482218,7729.516811,9329.782860,1915.881592,572.587158,1245.104614,12713.541221,573.441650,2479.612793,-3995.134521,1515.415039,1161.242432,4182.407715,-54.214321,3071.778076,1807.821533,2860.135498
4,20200525_4,68156.374202,18571.400456,802.975708,5664.883301,427.160889,807.170166,27592.610521,1027.783936,3625.424316,1848.052979,8937.720156,1860.545166,7593.783691,2874.432129,5232.925293,9535.824921,3880.500000,9692.408203,18426.507984,15664.580082,10140.423371,1769.410522,458.950897,1475.978882,27627.053595,265.749115,3299.099609,-145.335739,3262.548340,1233.912598,4366.298340,-142.910202,3088.685059,2015.936890,3346.034424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,20200531_19,273566.287858,81603.031114,7541.453613,22466.015625,7964.496094,5916.952637,116670.092004,5433.655273,16973.384766,6946.293945,57545.520572,11985.865234,25200.714844,15180.292969,31344.410156,47799.154425,11140.938477,27146.988281,59613.064096,82077.149704,62621.190825,17715.179688,11159.668945,7433.266602,136473.399940,6100.631836,14036.731445,21327.816406,16655.294922,5940.698242,18955.529297,2481.410889,16625.714844,9874.910156,17492.767578
164,20200531_20,235823.717952,61961.252511,4873.117676,16742.283203,5078.955566,3823.622803,99136.472653,4113.562012,12829.137695,5300.130371,44645.508347,8791.423828,20710.177734,11387.496094,23669.882812,37984.473193,9483.796875,20878.990234,52333.118859,65484.377003,46245.265296,14429.649414,7433.250977,5051.760742,117426.009670,3840.825195,11614.768555,14978.179688,14312.148438,4776.474121,14361.234375,1244.152222,11722.857422,6792.846680,12015.164062
165,20200531_21,210494.409356,48602.275491,3406.068848,12936.874023,3655.207031,2654.446533,83856.155175,3287.847168,10073.829102,4152.170410,35485.693594,6634.718262,17332.843750,8649.197266,18103.896484,30052.455002,8075.554688,16748.791016,45704.710095,51633.737993,34873.075537,11189.827148,5134.538574,3806.634766,100727.975338,2552.077148,9656.945312,10597.099609,12572.494141,3752.931641,11452.981445,596.608521,8654.374023,5246.957520,8768.124023
166,20200531_22,174372.332740,38364.922181,2531.028320,10263.922852,2908.451660,2086.475098,65228.955086,2621.168213,8039.231445,3200.352

In [17]:
submission.to_csv('prediction.csv', index=False)